In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
import random
import numpy as np
import sys

In [2]:

filename = "el_quijote.txt"
text = open(filename, encoding="utf-8").read()
text = text.replace('\n', ' ').replace('\r', '').replace('-',' ').replace("!","").replace("¿","").replace("¡","").replace("?","").replace("(","").replace(")","")
text = text.replace(': ',' ').replace(':',' ').replace('. ',' ').replace('.',' ').replace(',',' ').replace('<','').replace('>','')
text = text.replace('«','').replace('»','')
text = text.replace('[',' ').replace(']',' ').replace("'","").replace('"','').replace(';','').replace('-',' ').replace('  ',' ')
text = text.lower()
print(text[:200])

 capitulo 1 que trata de la condicion y ejercicio del famoso hidalgo d quijote de la mancha en un lugar de la mancha de cuyo nombre no quiero acordarme no ha mucho tiempo que vivia un hidalgo de los d


## Data Pre-Processing

In [4]:
maxlen = 60 #extract sequences of length 60
step = 3
sentences = []	#holds extracted sequences
next_chars = [] #holds the targets
for i in range(0, len(text)-maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+1:i+maxlen+1])
print(sentences[:5])
print(next_chars[:5])

[' capitulo 1 que trata de la condicion y ejercicio del famoso', 'pitulo 1 que trata de la condicion y ejercicio del famoso hi', 'ulo 1 que trata de la condicion y ejercicio del famoso hidal', ' 1 que trata de la condicion y ejercicio del famoso hidalgo ', 'que trata de la condicion y ejercicio del famoso hidalgo d q']
['capitulo 1 que trata de la condicion y ejercicio del famoso ', 'itulo 1 que trata de la condicion y ejercicio del famoso hid', 'lo 1 que trata de la condicion y ejercicio del famoso hidalg', '1 que trata de la condicion y ejercicio del famoso hidalgo d', 'ue trata de la condicion y ejercicio del famoso hidalgo d qu']


In [5]:
#VECTORIZATION
chars = sorted(set(text))
char_indices = dict((char, chars.index(char)) for char in chars)
print(char_indices)

{' ': 0, '0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10, 'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15, 'f': 16, 'g': 17, 'h': 18, 'i': 19, 'j': 20, 'k': 21, 'l': 22, 'm': 23, 'n': 24, 'o': 25, 'p': 26, 'q': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'x': 33, 'y': 34, 'z': 35, '̃': 36, '̈': 37}


In [6]:
for i, sentence in enumerate(sentences[0:100]):
    print("Seq:{} -> Sentencia: {} -> NextChar: {}".format(i,sentence,next_chars[i]))

Seq:0 -> Sentencia:  capitulo 1 que trata de la condicion y ejercicio del famoso -> NextChar: capitulo 1 que trata de la condicion y ejercicio del famoso 
Seq:1 -> Sentencia: pitulo 1 que trata de la condicion y ejercicio del famoso hi -> NextChar: itulo 1 que trata de la condicion y ejercicio del famoso hid
Seq:2 -> Sentencia: ulo 1 que trata de la condicion y ejercicio del famoso hidal -> NextChar: lo 1 que trata de la condicion y ejercicio del famoso hidalg
Seq:3 -> Sentencia:  1 que trata de la condicion y ejercicio del famoso hidalgo  -> NextChar: 1 que trata de la condicion y ejercicio del famoso hidalgo d
Seq:4 -> Sentencia: que trata de la condicion y ejercicio del famoso hidalgo d q -> NextChar: ue trata de la condicion y ejercicio del famoso hidalgo d qu
Seq:5 -> Sentencia:  trata de la condicion y ejercicio del famoso hidalgo d quij -> NextChar: trata de la condicion y ejercicio del famoso hidalgo d quijo
Seq:6 -> Sentencia: ata de la condicion y ejercicio del famoso hidalgo

In [7]:
print(sentences[0])
for t, char in enumerate(sentences[0]):        
        print(char," -> ",char_indices[char])

 capitulo 1 que trata de la condicion y ejercicio del famoso
   ->  0
c  ->  13
a  ->  11
p  ->  26
i  ->  19
t  ->  30
u  ->  31
l  ->  22
o  ->  25
   ->  0
1  ->  2
   ->  0
q  ->  27
u  ->  31
e  ->  15
   ->  0
t  ->  30
r  ->  28
a  ->  11
t  ->  30
a  ->  11
   ->  0
d  ->  14
e  ->  15
   ->  0
l  ->  22
a  ->  11
   ->  0
c  ->  13
o  ->  25
n  ->  24
d  ->  14
i  ->  19
c  ->  13
i  ->  19
o  ->  25
n  ->  24
   ->  0
y  ->  34
   ->  0
e  ->  15
j  ->  20
e  ->  15
r  ->  28
c  ->  13
i  ->  19
c  ->  13
i  ->  19
o  ->  25
   ->  0
d  ->  14
e  ->  15
l  ->  22
   ->  0
f  ->  16
a  ->  11
m  ->  23
o  ->  25
s  ->  29
o  ->  25


In [8]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=int)
y = np.zeros((len(sentences), len(chars)), dtype=int)
for i, sentence in enumerate(sentences):
    y[i, char_indices[next_chars[i]]] = 1
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1

KeyError: 'capitulo 1 que trata de la condicion y ejercicio del famoso '

In [43]:
print(x[0][0])

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]


In [44]:
print("x -> {} x {} x {}".format(len(x), len(x[0]), len(x[0][0])))
print("y -> {} x {}".format(len(y),len(y[0])))

x -> 133898 x 60 x 38
y -> 133898 x 38


## Building the Network

In [45]:
model = tf.keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation="softmax"))

In [46]:
opt = tf.keras.optimizers.RMSprop(learning_rate=0.1)
model.compile(loss="categorical_crossentropy", optimizer=opt)

In [47]:
def sample_next_char(preds, temperature=0.2):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [48]:
for epoch in range(1, 60):
    print("epoch ", epoch)
    model.fit(x, y, batch_size=128)
    # Selección de un texto de semilla aleatorio
    start_index = random.randint(0, len(x) - maxlen - 1)
    seed_text = text[start_index:start_index + maxlen - 1]
    print("--- Generating with seed: " , seed_text)
    generated_text = seed_text
    for i in range(100):
        sampled = np.zeros((1, maxlen, len(chars)), dtype=int)
        for i in range(len(generated_text)):
            charIndex = char_indices[generated_text[i]]
            sampled[0,i,charIndex] = 1        
        preds = model.predict(sampled,verbose = 0)        
        next_index = sample_next_char(preds[0])
        next_char = chars[next_index]
        generated_text = generated_text[2:] + next_char
        #print(generated_text)
        sys.stdout.write(next_char)

epoch  1
1047/1047 [==============================] - 118s 110ms/step - loss: 2.7155
--- Generating with seed:  go y si no llevarlos al corral y alli se hara la hoguera y 
gvvhvgvvtgvggghvgggvhvhgdgggvgvggvdggdvdggvvdggvvdggggdvhvtvggggdvvtggtvvvvggggghggghvgggggtghvdgdggepoch  2
1047/1047 [==============================] - 118s 113ms/step - loss: 1.8485
--- Generating with seed:  como lo dejo mandado grisostomo y sobre esto anda el pueblo
                         p         t             t                                      p           epoch  3
1047/1047 [==============================] - 123s 118ms/step - loss: 10.4907
--- Generating with seed:   del mas pequeño dolor que tenga si ya no se entiende tamb
rd

<ipython-input-47-3d191d290153>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


ny ynasaa tcsl snpcrs yprmsnsoayisidl nrala pf araojr a pexj laaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaepoch  4
1047/1047 [==============================] - 121s 115ms/step - loss: 22.3270
--- Generating with seed:  a de quemarlos que de echar una tela por grande y delgada q
leecdeideeegle deeqeceelcejeejeeeedeetleectecednẽleecectte le le lee lene le le le le le le le le lepoch  5
1047/1047 [==============================] - 122s 116ms/step - loss: 6.7279
--- Generating with seed:   de esta historia esta batalla disculpandose que no hallo m
ooeeooo sjmoyosoe sses ys oeoevoeyso oomo sysooooogxoeoeoo eose o e e e o e e o ese e e e e e e e e epoch  6
1047/1047 [==============================] - 117s 112ms/step - loss: 4.0230
--- Generating with seed:  a contrahecha celada que traia atada con unas cintas verdes
   ooppe o o o dp  pp     pp  opop oe p   ddp p  p      dp p o   p   p     p p p        p       p   epoch  7
1047/1047 [==============================] - 116s 111ms/step - loss: 3.6588


KeyboardInterrupt: 